In [1]:
import bs4
import requests
import re
import json
import pandas as pd

from os import path
from tqdm import tqdm_notebook
from time import sleep

In [130]:
base_url = 'https://www.cvk.gov.ua/pls/vp2019/'
page = requests.get(path.join(base_url,'wp030pt001f01=719.html')).text
soup = bs4.BeautifulSoup(page,'html.parser')

district_links = []
for item in soup.find_all('tr', class_="tr"):
    district_links.append(item.find(class_ = 'a1').get('href'))
        
district_links[:10]

['wp024npt001f01=719pid100=5pt005f01=11.html',
 'wp024npt001f01=719pid100=5pt005f01=12.html',
 'wp024npt001f01=719pid100=5pt005f01=13.html',
 'wp024npt001f01=719pid100=5pt005f01=14.html',
 'wp024npt001f01=719pid100=5pt005f01=15.html',
 'wp024npt001f01=719pid100=5pt005f01=16.html',
 'wp024npt001f01=719pid100=5pt005f01=17.html',
 'wp024npt001f01=719pid100=5pt005f01=18.html',
 'wp024npt001f01=719pid100=7pt005f01=19.html',
 'wp024npt001f01=719pid100=7pt005f01=20.html']

In [131]:
def parse_row(results):
    for row in results:
        for index,column in enumerate(row.find_all('td')[:2]):
            if index == 0:
                district_number = column.text.replace(u'\n', u'')
            elif index == 1:
                district_location = column.text.replace(u'\n', u'')
        records.append([district_general,district_number,district_location])

records = []

for district_link in tqdm_notebook(district_links):
    if district_link == 'wp045pt001f01=719.html':
        page = requests.get(path.join(base_url,district_link)).text
        soup = bs4.BeautifulSoup(page,'html5lib')
        fd_links = [link.get('href') for link in soup.find_all(class_="pure-table")[1].find_all(class_='a1')]

        for fd_link in fd_links:
            page = requests.get(path.join(base_url,fd_link)).text
            soup = bs4.BeautifulSoup(page,'html5lib')
            sleep(0.2)

            district_general = soup.find('h3').text
            results_raw = soup.find(class_='pure-table w100').find('tbody')
            parse_row(results_raw.find_all('tr'))
            
    else:
        page = requests.get(path.join(base_url,district_link)).text
        soup = bs4.BeautifulSoup(page,'html5lib')
        inner_link = soup.find(class_="fas fa-2x fa-warehouse").parent.get('href')

        page = requests.get(path.join(base_url,inner_link)).text
        soup = bs4.BeautifulSoup(page,'html5lib')
        sleep(0.2)

        district_general = soup.find(class_='pure-u-1 cntr').text.replace(u'\xa0', u'').replace(u'\n', u'')
        results_raw = soup.find(class_='pure-table pure-table-horizontal').find('tbody')
        parse_row(results_raw.find_all('tr')[1:])
    
colnames = ['district_general','district_number','district_location']
df_locations = pd.DataFrame.from_records(records,columns=colnames)
print(df_locations.shape)
df_locations.head()


(29982, 3)


,district_general,district_number,district_location
0,"Вінницька область, Територіальний виборчий окр...",050130,"Школа-ліцей, спортивна зала"
1,"Вінницька область, Територіальний виборчий окр...",050131,"ДНЗ ""Казка"", спортивна зала"
2,"Вінницька область, Територіальний виборчий окр...",050132,"ПСП ""Агрофірма Батьківщина"", приіщення контори"
3,"Вінницька область, Територіальний виборчий окр...",050133,"школа, спортзал"
4,"Вінницька область, Територіальний виборчий окр...",050134,"звіргосподарство, кабінет головного бухгалтера"


In [132]:
base_url = 'https://www.cvk.gov.ua/pls/vp2019/'
page = requests.get(path.join(base_url,'wp335pt001f01=720.html')).text
soup = bs4.BeautifulSoup(page,'html5lib')
links = [link_raw.get('href') for link_raw in soup.find_all(class_='a1')]

districts = [int(link.split('=')[-1].replace('.html','')) for link in links]
links_and_districts = [i for i in zip(districts,links)]
links_and_districts = links_and_districts[1:]

    
districts_processed = [int(item.text.replace('\n',''))
                       for item in soup.find_all(class_='cntr b') 
                       if (item.get('style')!='border:1px solid #cbcbcb')&(item.text != 'ЗВО\n')]

print(len(districts_processed))
print(sum(districts_processed))

links_and_districts[:10]

200
29608


[(11, 'wp336pt001f01=720pt005f01=11.html'),
 (12, 'wp336pt001f01=720pt005f01=12.html'),
 (13, 'wp336pt001f01=720pt005f01=13.html'),
 (14, 'wp336pt001f01=720pt005f01=14.html'),
 (15, 'wp336pt001f01=720pt005f01=15.html'),
 (16, 'wp336pt001f01=720pt005f01=16.html'),
 (17, 'wp336pt001f01=720pt005f01=17.html'),
 (18, 'wp336pt001f01=720pt005f01=18.html'),
 (19, 'wp336pt001f01=720pt005f01=19.html'),
 (20, 'wp336pt001f01=720pt005f01=20.html')]

In [133]:
colvalues = []
district_gen_numeric = []

for link in tqdm_notebook(links_and_districts):
    page = requests.get(path.join(base_url,link[1])).text
    soup = bs4.BeautifulSoup(page,'html5lib')
    results_raw = soup.find(class_='pure-table pure-table-horizontal w100')
    results_raw.find('thead').find_all('th')[0].text
    colnames = [colname.text.replace(u'\n', u'') for colname in results_raw.find('thead').find_all('th')]
    for row in results_raw.find_all('tr')[1:]:
        colvalues_row = []
        for colvalue in row.find_all('td'):
            colvalues_row.append(colvalue.text.replace(u'\n', u''))
        colvalues.append(colvalues_row)
        district_gen_numeric.append(link[0])
    sleep(0.2)

df_results = pd.DataFrame.from_records(colvalues,columns=colnames)
df_results['district_gen_numeric'] = pd.Series(district_gen_numeric)
print(sum(districts_processed)==df_results.shape[0])
print(df_results.shape)
df_results.head()


True
(29608, 15)


,№ ВД,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.","7.Загальна к-сть виборців, які отримали бюлетені","8.К-сть бюлетенів, що не підлягають врахуванню","9.К-сть виборців, які взяли участь у голосуванні","10.К-сть бюлетенів, визнаних недійсними",ЗеленськийВолодимир,ПорошенкоПетро,Дата та час протоколу,district_gen_numeric
0,050130,1696,1694,103,602,1004,90,1094,0,1094,16,579,499,22.04.2019 00:30,11
1,050131,2128,2095,15,822,1295,11,1306,0,1306,28,751,527,22.04.2019 00:20,11
2,050132,1296,1281,4,484,808,4,812,0,812,14,471,327,22.04.2019 01:15,11
3,050133,2448,2422,14,933,1504,11,1515,0,1515,37,769,709,22.04.2019 01:30,11
4,050134,240,222,0,108,132,0,132,0,132,0,77,55,21.04.2019 22:10,11


In [134]:
df_locations['district_gen_numeric'] = df_locations['district_general'].map(lambda x: int(x.split('№')[-1]) if len(x.split('№')[-1])<4 else 226)
df_locations['district_number'] = df_locations.loc[:,'district_number'].map(lambda x: x.replace(' ',''))
df_all = df_results.rename(columns={'№ ВД':'district_number'}).merge(df_locations,on=['district_number','district_gen_numeric'])
df_all = df_all[['district_number','district_gen_numeric', 'district_general','district_location'] + list(df_all.columns[1:-3])]
df_all['2.К-сть виборців, внесених до списку'] = pd.to_numeric(df_all['2.К-сть виборців, внесених до списку'])
print(df_all.shape)
df_all.head()

(29608, 17)


,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.","7.Загальна к-сть виборців, які отримали бюлетені","8.К-сть бюлетенів, що не підлягають врахуванню","9.К-сть виборців, які взяли участь у голосуванні","10.К-сть бюлетенів, визнаних недійсними",ЗеленськийВолодимир,ПорошенкоПетро,Дата та час протоколу
0,050130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1696,1694,103,602,1004,90,1094,0,1094,16,579,499,22.04.2019 00:30
1,050131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2128,2095,15,822,1295,11,1306,0,1306,28,751,527,22.04.2019 00:20
2,050132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1296,1281,4,484,808,4,812,0,812,14,471,327,22.04.2019 01:15
3,050133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2448,2422,14,933,1504,11,1515,0,1515,37,769,709,22.04.2019 01:30
4,050134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",240,222,0,108,132,0,132,0,132,0,77,55,21.04.2019 22:10


In [135]:
dtype_dic = {'district_number': str}
df_round1 = pd.read_csv('00_data/elections_round1_v1.csv',dtype = dtype_dic)
df_round1 = df_round1[['district_number','district_gen_numeric','2.К-сть виборців, внесених до списку']]
df_round1.head()

,district_number,district_gen_numeric,"2.К-сть виборців, внесених до списку"
0,050130,11,1686
1,050131,11,2109
2,050132,11,1282
3,050133,11,2434
4,050134,11,225


In [136]:
lost_distr = df_round1.merge(df_locations,on=['district_number','district_gen_numeric'],how='left')
lost_distr = lost_distr.loc[lost_distr['district_location'].isnull()]
print(lost_distr['2.К-сть виборців, внесених до списку'].sum())
lost_distr

112


,district_number,district_gen_numeric,"2.К-сть виборців, внесених до списку",district_general,district_location
463,051667,13,51,NaN,NaN
17476,000004,143,10,NaN,NaN
17480,000010,143,11,NaN,NaN
17483,000014,143,8,NaN,NaN
17484,000015,143,12,NaN,NaN
17487,000018,143,12,NaN,NaN
17489,000021,143,8,NaN,NaN


In [137]:
df_all_compare = df_all.merge(df_round1, on=['district_number','district_gen_numeric'])
df_all_compare['reg_voters_change'] = df_all_compare['2.К-сть виборців, внесених до списку_x'] - df_all_compare['2.К-сть виборців, внесених до списку_y']
df_all_compare['reg_voters_change_abs'] = df_all_compare['reg_voters_change'].map(lambda x: abs(x))
print(df_all_compare['reg_voters_change'].sum())
print(df_all_compare['reg_voters_change_abs'].sum())
print(df_all_compare['reg_voters_change_abs'].sum()/29657746)
print(df_all_compare.shape)
df_all_compare.head()

51636
232162
0.007828039258276742
(29608, 20)


,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку_x","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.","7.Загальна к-сть виборців, які отримали бюлетені","8.К-сть бюлетенів, що не підлягають врахуванню","9.К-сть виборців, які взяли участь у голосуванні","10.К-сть бюлетенів, визнаних недійсними",ЗеленськийВолодимир,ПорошенкоПетро,Дата та час протоколу,"2.К-сть виборців, внесених до списку_y",reg_voters_change,reg_voters_change_abs
0,050130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1696,1694,103,602,1004,90,1094,0,1094,16,579,499,22.04.2019 00:30,1686,8,8
1,050131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2128,2095,15,822,1295,11,1306,0,1306,28,751,527,22.04.2019 00:20,2109,-14,14
2,050132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1296,1281,4,484,808,4,812,0,812,14,471,327,22.04.2019 01:15,1282,-1,1
3,050133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2448,2422,14,933,1504,11,1515,0,1515,37,769,709,22.04.2019 01:30,2434,-12,12
4,050134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",240,222,0,108,132,0,132,0,132,0,77,55,21.04.2019 22:10,225,-3,3


In [138]:
df_all_compare.to_csv('00_data/elections_round2_v3.csv',index=False)